In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import sklearn
#import statsmodels.api as sm
#import statsmodels.formula.api as smf
import imblearn as imb
from datetime import datetime

In [184]:
birds = pd.read_csv('birds_final.csv')

We read in our csv. This corresponds to the pre-processed eBird data for a selection of 24 different species within the Amazonas state of Brazil (specifically filtered down to the four sites we selected) between the years 2012-2021 (inclusive). See the notebook 'eBird_preprocessing' for details on the preprocessing steps.

In [59]:
clim_raw = pd.read_csv('clim_raw.csv')

We will also need some of the raw climate data, contained in the file 'clim_raw.csv'

In [185]:
birds.head()

,checklist_id,state_code,locality_id,latitude,longitude,observation_date,protocol_type,duration_minutes,effort_distance_km,number_observers,...,Shape_Area,Site_ID,X_coord,Y_coord,tmax,tmin,prec,EVI,umd_tree_cover_loss_ha,umd_tree_cover_loss_from_fires_ha
0,S130090237,BR-AM,L2872043,-3.062795,-60.107907,2012-02-11,Traveling,60.0,0.5,6.0,...,1.867389e+10,69.0,-59.705321,-3.261773,29.0,23.19,291.3,0.464591,163604.2673,25449.43001
1,S130090237,BR-AM,L2872043,-3.062795,-60.107907,2012-02-11,Traveling,60.0,0.5,6.0,...,1.867389e+10,69.0,-59.705321,-3.261773,29.0,23.19,291.3,0.464591,163604.2673,25449.43001
2,S130090237,BR-AM,L2872043,-3.062795,-60.107907,2012-02-11,Traveling,60.0,0.5,6.0,...,1.867389e+10,69.0,-59.705321,-3.261773,29.0,23.19,291.3,0.464591,163604.2673,25449.43001
3,S130090237,BR-AM,L2872043,-3.062795,-60.107907,2012-02-11,Traveling,60.0,0.5,6.0,...,1.867389e+10,69.0,-59.705321,-3.261773,29.0,23.19,291.3,0.464591,163604.2673,25449.43001
4,S130090237,BR-AM,L2872043,-3.062795,-60.107907,2012-02-11,Traveling,60.0,0.5,6.0,...,1.867389e+10,69.0,-59.705321,-3.261773,29.0,23.19,291.3,0.464591,163604.2673,25449.43001


In [186]:
birds.columns

Index(['checklist_id', 'state_code', 'locality_id', 'latitude', 'longitude',
       'observation_date', 'protocol_type', 'duration_minutes',
       'effort_distance_km', 'number_observers', 'scientific_name',
       'observation_count', 'species_observed', 'effort_hours',
       'effort_speed_kmph', 'hours_of_day', 'year', 'day_of_year', 'Family',
       'Order_', 'Habitat', 'Habitat_Density', 'Migration', 'Trophic_Level',
       'Trophic_Niche', 'Primary_Lifestyle', 'Guild', 'Sp_Gen', 'SNST',
       'STRAT', 'REL', 'Unique_ID', 'Site_ID_1', 'OBJECTID', 'Shape_Length',
       'Shape_Area', 'Site_ID', 'X_coord', 'Y_coord', 'tmax', 'tmin', 'prec',
       'EVI', 'umd_tree_cover_loss_ha', 'umd_tree_cover_loss_from_fires_ha'],
      dtype='object')

In [75]:
clim_raw

,Unnamed: 0,Lat,Long,prec,Date,Site_ID,tmin,tmax
0,0,-3.25,-61.25,434.34,2000-01-01,68,22.50,30.0
1,1,-3.25,-61.25,339.41,2000-02-01,68,23.00,30.0
2,2,-3.25,-61.25,364.26,2000-03-01,68,23.00,30.0
3,3,-3.25,-61.25,420.14,2000-04-01,68,23.00,30.0
4,4,-3.25,-61.25,278.97,2000-05-01,68,23.00,31.0
...,...,...,...,...,...,...,...,...
1051,1051,-2.25,-59.75,81.55,2019-08-01,79,24.00,32.0
1052,1052,-2.25,-59.75,210.88,2019-09-01,79,24.00,33.0
1053,1053,-2.25,-59.75,264.12,2019-10-01,79,24.00,32.0
1054,1054,-2.25,-59.75,165.46,2019-11-01,79,24.19,32.0


In [187]:
# Quick modification to convert the 'observation_date' column of birds and the 'Date' column of clim_raw to
# DateTime objects
birds['observation_date'] = pd.to_datetime(birds['observation_date'], format='%Y-%m-%d')
clim_raw['Date'] = pd.to_datetime(clim_raw['Date'], format='%Y-%m-%d')

Of the current columns in the dataframe, below are lists of those variables (covariates) we are interested in, divided into three categories. 
-  occupancy_covs: These are occupancy covariates, features that could influence whether or not a species occupies a site on an ecological level (e.g. 'umd_tree_cover_loss_ha' tells us about the tree-cover loss. Fewer trees might make it less likely that a given species occupies a particular site. 
<br>

- trait_covs: These are trait covariates, features describing the traits of a particular species (e.g. 'Trophic_Niche' could take the value 'Omnivore.' 
<br>

- effort_covs: These are effort covariates, features related to the effort level and circumstances of a particular observation outing (e.g. 'effort_hours' tells us how long the observers spent looking for birds on a given day)
<br>

Note that we are intereseted in modeling the effects of the occupancy covariates over time, so we will have to take into account the temporal nature of those features.

In [188]:
occupancy_covs = ['prec', 'tmax', 'tmin', 'EVI','umd_tree_cover_loss_ha', 'umd_tree_cover_loss_from_fires_ha']
trait_covs = ['Trophic_Niche', 'Sp_Gen', 'SNST', 'REL', 'Primary_Lifestyle']
effort_covs = ['observation_date', 'effort_distance_km', 'number_observers', 'effort_hours', 'year', 
               'day_of_year', 'hours_of_day']

In [194]:
# Trim down the dataframe to just columns we want
all_cols = occupancy_covs + trait_covs + effort_covs + ['checklist_id', 'scientific_name', 
                                                        'species_observed', 'Site_ID']
birds = birds[all_cols]

In [195]:
# Not really using this right now - safe to ignore!

# Function to sample our eBird data over a specified time period for specified sites at one of several
# specified frequencies. 

def ebird_sampler(df, species, sample_size = 2, year_start = 2015, year_stop = 2021, 
                  sites = [68,69,78,79], frequency = 'biannual'):
    """
    Arguments:
        df: dataframe from which to sample
        species: scientific_name of species to select
        sample_size: number of samples to draw
        year_start: first year of interst
        year_stop: last year of interst
        sites: list of site (numbers) to consider
        frequency: options are 'annual', 'biannual', 'quarterly', 'monthly'
    
    Returns: dataframe with sampled eBird data at specififed frequency. Will raise an exception if there are 
    not enough eBird checklists in some period at the frequency specified. 
    """
    years = np.arange(year_start, year_stop+1, 1)
    # Set the periods for sub-annual frequency sampling
    if frequency == 'biannual':
        half0 = [1,2,3,4,5,6]
        half1 = [7,8,9,10,11,12]
        sub = [half0, half1]
    if frequency == 'quarterly':
        quarter0 = [1,2,3]
        quarter1 = [4,5,6]
        quarter2 = [7,8,9]
        quarter3 = [10,11,12]
        sub = [quarter0,quarter1,quarter2,quarter3]
    if frequency == 'monthly':
        sub = [([month]) for month in range(1,13)]
    
    # Set up an empty dataframe (with the same columns as the original data) which will hold our sampled data.
    birds_sampled = df.sample(0).copy()
    for sitenum in sites:
        year_counter = 0
        for year in years: # Loop through years
            if frequency == 'annual':
                # Set the criteria - for annual frequency we only need to discriminate on the species, site, and year
                criteria = (df.scientific_name == species) & (df.Site_ID == sitenum) & (df.observation_date.dt.year == year)
                # Get the subset of the data meeting criteria
                df_subset = df.loc[criteria]
                if  len(df_subset) == 0:
                    raise Exception('No checklists at site' + str(sitenum) + ' for ' + str(year))
                # Find the unique checklists meeting the above criteria and choose (sample_size) of them
                if len(df_subset.checklist_id.unique()) < sample_size:
                    raise Exception('Insufficient checklists (' + str(len(df_subset.checklist_id.unique())) + ') at site' + str(sitenum) + ' for ' + str(year))
                checks_picked = np.random.choice(df_subset.checklist_id.unique(), sample_size, replace = False)
                # Get the rows of the dataframe meeting criteria and corresponding to the chosen checklist_ids
                sample_subset = df_subset.loc[df_subset.checklist_id.isin(checks_picked)].copy()
                # Add a column for the time interval (for later time-series shifting purposes)
                sample_subset['time_interval'] = year_counter*np.ones(len(sample_subset))
                # Add the data sample to our master dataframe
                birds_sampled = pd.concat([birds_sampled, sample_subset])
            # Next handle all other sampling frequencies
            else: 
                period_counter = 0
                for period in sub: # Loop through the periods corresponding to our sampling frequency
                    # Set the criteria - for the sub-annual frequencies we discriminate on month in addition to year and site number 
                    criteria = (df.scientific_name == species) & (df.Site_ID == sitenum) & (df.observation_date.dt.year == year) & (df.observation_date.dt.month.isin(period))
                    # Get the subset of the dataset meeting criteria
                    df_subset = df.loc[criteria]
                    if  len(df_subset) == 0:
                        raise Exception('No checklists at site' + str(sitenum) + ' for ' + str(year) + ' period ' 
                                        + str(sub.index(period)) + ' at frequency ' + str(frequency))
                    # Find the unique checklists meeting the above criteria and choose (sample_size) of them
                    if len(df_subset.checklist_id.unique()) < sample_size:
                        raise Exception('Insufficient checklists (' + str(len(df_subset.checklist_id.unique())) + ') at site' + str(sitenum) + ' for ' + str(year) + ' period ' 
                                        + str(sub.index(period)) + ' at frequency ' + str(frequency))
                    checks_picked = np.random.choice(df_subset.checklist_id.unique(), sample_size, replace = False)
                    # Get the rows of the dataframe meeting criteria and corresponding to the chosen checklist_ids
                    sample_subset = df_subset.loc[df_subset.checklist_id.isin(checks_picked)].copy()
                    # Add a column for the time interval (for later time-series shifting purposes)
                    sample_subset['time_interval'] = (year_counter+period_counter)*np.ones(len(sample_subset))
                    # Add the data sample to our master dataframe
                    birds_sampled = pd.concat([birds_sampled, sample_subset])
                    period_counter += 1
            year_counter += period_counter
    return birds_sampled

In [196]:
# Not really using this right now - safe to ignore!
# Function to see how many unique checklists we have for periods and sites at a given frequency

def ebird_numchecks(df, species, year_start = 2015, year_stop = 2021, frequency = 'biannual'):
    """
    # df: dataframe from which to sample
    # species: scientific_name of species to select
    # sample_size: number of samples to draw
    # year_start: first year of interst
    # year_stop: last year of interst
    # sites: list of site (numbers) to consider
    # frequency: options are 'annual', 'biannual', 'quarterly', 'monthly'
    """
    years = np.arange(year_start, year_stop+1, 1)
    
    
    if frequency == 'annual':
        holder_arr = np.zeros((len(years), 5))
        col_names = ['year', 'checks_at_68', 'checks_at_69', 'checks_at_78', 'checks_at_79']
        i = 0 # year counter
        for year in years: # Loop through years
            criteria_68 = (df.scientific_name == species) & (df.Site_ID == 68) & (df.observation_date.dt.year == year)
            criteria_69 = (df.scientific_name == species) & (df.Site_ID == 69) & (df.observation_date.dt.year == year)                
            criteria_78 = (df.scientific_name == species) & (df.Site_ID == 78) & (df.observation_date.dt.year == year)
            criteria_79 = (df.scientific_name == species) & (df.Site_ID == 79) & (df.observation_date.dt.year == year)
            checks_68 = len(df.loc[criteria_68,'checklist_id'].unique())
            checks_69 = len(df.loc[criteria_69,'checklist_id'].unique())
            checks_78 = len(df.loc[criteria_78,'checklist_id'].unique())
            checks_79 = len(df.loc[criteria_79,'checklist_id'].unique())
            holder_arr[i,0] = year
            holder_arr[i,1] = checks_68
            holder_arr[i,2] = checks_69
            holder_arr[i,3] = checks_78
            holder_arr[i,4] = checks_79
            i += 1
        check_df = pd.DataFrame(holder_arr, columns = col_names)

    if frequency == 'biannual':
        half0 = [1,2,3,4,5,6]
        half1 = [7,8,9,10,11,12]
        sub = [half0, half1]
        holder_arr = np.zeros((len(years)*len(sub), 6))
        col_names = ['year', 'period', 'checks_at_68', 'checks_at_69', 'checks_at_78', 'checks_at_79']
        i = 0 # year counter
        for year in years: # Loop through years
            j = 0 # period counter
            for period in sub:
                criteria_68 = (df.scientific_name == species) & (df.Site_ID == 68) & (df.observation_date.dt.year == year) & (df.observation_date.dt.month.isin(period))
                criteria_69 = (df.scientific_name == species) & (df.Site_ID == 69) & (df.observation_date.dt.year == year) & (df.observation_date.dt.month.isin(period))
                criteria_78 = (df.scientific_name == species) & (df.Site_ID == 78) & (df.observation_date.dt.year == year) & (df.observation_date.dt.month.isin(period))
                criteria_79 = (df.scientific_name == species) & (df.Site_ID == 79) & (df.observation_date.dt.year == year) & (df.observation_date.dt.month.isin(period))
                checks_68 = len(df.loc[criteria_68,'checklist_id'].unique())
                checks_69 = len(df.loc[criteria_69,'checklist_id'].unique())
                checks_78 = len(df.loc[criteria_78,'checklist_id'].unique())
                checks_79 = len(df.loc[criteria_79,'checklist_id'].unique())
                holder_arr[i+j,0] = year
                holder_arr[i+j,1] = sub.index(period)
                holder_arr[i+j,2] = checks_68
                holder_arr[i+j,3] = checks_69
                holder_arr[i+j,4] = checks_78
                holder_arr[i+j,5] = checks_79
                j += 1
            i += j
        check_df = pd.DataFrame(holder_arr, columns = col_names)
        
    if frequency == 'quarterly':
        quarter0 = [1,2,3]
        quarter1 = [4,5,6]
        quarter2 = [7,8,9]
        quarter3 = [10,11,12]
        sub = [quarter0,quarter1,quarter2,quarter3]
        holder_arr = np.zeros((len(years)*len(sub), 6))
        col_names = ['year', 'period', 'checks_at_68', 'checks_at_69', 'checks_at_78', 'checks_at_79']
        i = 0 # year counter
        for year in years: # Loop through years
            j = 0 # period counter
            for period in sub:
                criteria_68 = (df.scientific_name == species) & (df.Site_ID == 68) & (df.observation_date.dt.year == year) & (df.observation_date.dt.month.isin(period))
                criteria_69 = (df.scientific_name == species) & (df.Site_ID == 69) & (df.observation_date.dt.year == year) & (df.observation_date.dt.month.isin(period))                    
                criteria_78 = (df.scientific_name == species) & (df.Site_ID == 78) & (df.observation_date.dt.year == year) & (df.observation_date.dt.month.isin(period))
                criteria_79 = (df.scientific_name == species) & (df.Site_ID == 79) & (df.observation_date.dt.year == year) & (df.observation_date.dt.month.isin(period))
                checks_68 = len(df.loc[criteria_68,'checklist_id'].unique())
                checks_69 = len(df.loc[criteria_69,'checklist_id'].unique())
                checks_78 = len(df.loc[criteria_78,'checklist_id'].unique())
                checks_79 = len(df.loc[criteria_79,'checklist_id'].unique())
                holder_arr[i+j,0] = year
                holder_arr[i+j,1] = sub.index(period)
                holder_arr[i+j,2] = checks_68
                holder_arr[i+j,3] = checks_69
                holder_arr[i+j,4] = checks_78
                holder_arr[i+j,5] = checks_79
                j += 1
            i += j
        check_df = pd.DataFrame(holder_arr, columns = col_names)
        
    if frequency == 'monthly':
        sub = [([month]) for month in range(1,13)]
        holder_arr = np.zeros((len(years)*len(sub), 6))
        col_names = ['year', 'period', 'checks_at_68', 'checks_at_69', 'checks_at_78', 'checks_at_79']
        i = 0 # year counter
        for year in years: # Loop through years
            j = 0 # period counter
            for period in sub:
                criteria_68 = (df.scientific_name == species) & (df.Site_ID == 68) & (df.observation_date.dt.year == year) & (df.observation_date.dt.month.isin(period))
                criteria_69 = (df.scientific_name == species) & (df.Site_ID == 69) & (df.observation_date.dt.year == year) & (df.observation_date.dt.month.isin(period))
                criteria_78 = (df.scientific_name == species) & (df.Site_ID == 78) & (df.observation_date.dt.year == year) & (df.observation_date.dt.month.isin(period))
                criteria_79 = (df.scientific_name == species) & (df.Site_ID == 79) & (df.observation_date.dt.year == year) & (df.observation_date.dt.month.isin(period))
                checks_68 = len(df.loc[criteria_68,'checklist_id'].unique())
                checks_69 = len(df.loc[criteria_69,'checklist_id'].unique())
                checks_78 = len(df.loc[criteria_78,'checklist_id'].unique())
                checks_79 = len(df.loc[criteria_79,'checklist_id'].unique())
                holder_arr[i+j,0] = year
                holder_arr[i+j,1] = sub.index(period)
                holder_arr[i+j,2] = checks_68
                holder_arr[i+j,3] = checks_69
                holder_arr[i+j,4] = checks_78
                holder_arr[i+j,5] = checks_79
                j += 1
            i += j
        check_df = pd.DataFrame(holder_arr, columns = col_names)

    return check_df

Our setup here is a supervised learning (classification) problem, not a pure time series problem. So we need to take the data that have a time series element (the occupancy covariates) and turn them into features for supervised learning. We will do this by creating shifted features, so that for a particular observation/checklist, we have a new feature holding the value of our occupancy covariate at the previous time step. Because the environmental and climate data we were able to obtain have different frequencies, the shifts for each of the features will be done at these different frequencies as well. Full details are below:

 - Shift 'EVI', 'umd_tree_cover_loss_ha', and 'umd_tree_cover_loss_from_fires_ha' by 1 year. The expectation is that past ecological conditions should influence whether or not a species presently occupies a site.
 - Shift 'tmax', 'tmin', and 'prec' by 1 month. Similar expectation to the above, that climate conditions in the past should influence whether or not a species presently occupies a site.
 - Shift species_observed by whatever period corresponds to the previous observation (note this could be as short as a single day or potentially much longer if no observations had been carried out recently). The expectation here is that we should have some kind of temporal auto-correlation. That is, the fact that a species was detected at a site the last time someone looked likely influences the probability that the species will be detected the next time someone looks.

So for a given eBird observation I will know:
- 'EVI', 'umd_tree_cover_loss_ha', and 'umd_tree_cover_loss_from_fires_ha' from the previous year
- 'tmax', 'tmin', and 'prec' for the previous month
- and occupancy status ('species_observed') for the last time the bird was looked for

Below are some functions to do these things for our data set. Note we will do this separately for each species we consider.

In [197]:
# Function to create new (shifted) columns corresponding to 'EVI', 'umd_tree_cover_loss_ha',
# and 'umd_tree_cover_loss_from_fires_ha' from the previous year
def yearly_cov_shift(df_in, year_start = 2012, year_stop = 2021):
    '''
    Takes our eBird dataframe and shifts the occupancy covariates for which data is yearly by 1 year. 
    Note these covariates are 'EVI', 'umd_tree_cover_loss_ha', and 'umd_tree_cover_loss_from_fires_ha'
    
    df_in: dataframe
    year_start: first year of data (will receive a shifted value of Nan since no data prior to this)
    year_stop: last year of data
    '''
    df = df_in.copy()
    covs = ['EVI', 'umd_tree_cover_loss_ha', 'umd_tree_cover_loss_from_fires_ha']
    sites = df.Site_ID.unique()
    for cov in covs:
        for site in sites:
            # Handle the first year, whose shifted column will be populated with nan
            df.loc[(df.observation_date.dt.year == year_start) & (df.Site_ID == site), cov+str('_prev_yr')] = np.nan
            for year in np.arange(year_start+1, year_stop + 1, 1):
                df.loc[(df.observation_date.dt.year == year) & (df.Site_ID == site), cov+str('_prev_yr')] = df.loc[(df.observation_date.dt.year == year - 1) & (df.Site_ID == site), cov].unique()[0]      
    return df

In [198]:
# Function to create new (shifted) columns corresponding to 'tmax', 'tmin', and 'prec' from the previous month.
# Note that this function requires the climate data in 'clim_raw'
def monthly_cov_shift(df_in, year_start = 2012, year_stop = 2021):
    '''
    Takes our eBird dataframe and shifts the occupancy covariates for which data is monthly by one month. 
    Note these covariates are 'tmax', 'tmin', and 'prec'
    Note - Relies on the additional dataframe 'clim_raw' which has the raw data for these three monthly covariates.
    The reason is that the eBird data has some months with no checklists. So even though we have already joined 
    the climate data to the main eBird data, the process of joining this monthly covariate data onto the eBird 
    dataframe only populates those months which do have checklists. But we still have access to the full scope of 
    monthly data for these covariates, even when there is no eBird checklist for a iven month. 
    So we use the raw covariate data contained in 'clim_raw' here.
    
    df_in: dataframe
    year_start: first year of data (will receive a shifted value of Nan since no data prior to this)
    year_stop: last year of data
    '''
    df = df_in.sort_values(by='observation_date').reset_index().copy() # Sort chronologically, reset indices, and copy
    covs = ['tmax', 'tmin', 'prec']
    sites = df.Site_ID.unique()
    for cov in covs:
        for site in sites:
            # Handle the first year
            # Find and handle the first month, whose shifted column will be populated with nan
            first_month = df.loc[(df.Site_ID == site) & (df.observation_date.dt.year == year_start), 'observation_date'].min().month
            df.loc[(df.Site_ID == site) & (df.observation_date.dt.year == year_start) & (df.observation_date.dt.month == first_month), cov+str('_prev_month')] = np.nan
            # Handle the rest of the months in the first year
            for month in np.arange(first_month+1, 12 + 1, 1):
                prev_cov = clim_raw.loc[(clim_raw.Site_ID == site) & (clim_raw.Date.dt.year == year_start) & (clim_raw.Date.dt.month == month-1), cov].unique()[0]
                df.loc[(df.Site_ID == site) & (df.observation_date.dt.year == year_start) & (df.observation_date.dt.month == month), cov+str('_prev_month')] = prev_cov   
            # Now move on to the subsequent years
            for year in np.arange(year_start+1, year_stop + 1, 1):
                for month in np.arange(1,12+1,1):
                    # Get the current and previous month/year (year necessary if the current month is January)
                    current_mo_yr = pd.to_datetime(str(month)+'-'+str(year))
                    if month == 1:
                        prev_mo_yr =  pd.to_datetime('12'+'-'+str(year-1))
                    elif month != 1:
                        prev_mo_yr =  pd.to_datetime(str(month-1)+'-'+str(year))
                    # Now set the shifted column for the current month equal to the equivalent column from the previous month in the 'clim_raw' dataframe
                    prev_cov = clim_raw.loc[(clim_raw.Site_ID == site) & (clim_raw.Date.dt.year == prev_mo_yr.year) & (clim_raw.Date.dt.month == prev_mo_yr.month), cov].unique()[0]
                    df.loc[(df.Site_ID == site) & (df.observation_date.dt.year == current_mo_yr.year) & (df.observation_date.dt.month == current_mo_yr.month), cov+str('_prev_month')] = prev_cov   
    return df

In [199]:
# Function to create a new (shifted) column corresponding to occupancy ('species_observed') 
# from the previous year. Note that this will not shift all the data by the same frequency since a particular 
# eBird checklist may have another eBird checklist
def occ_shift(df_in):
    '''
    Takes our eBird dataframe and shifts the occupancy variable 'species_observed' by 1 observation. Note
    that this is variable-frequency. The previous observation could have been the day before or it could have
    been the year before. 
    ASSUMES df_in ONLY CONTAINS ONE SPECIES
    Also assumes that there are no site overlaps between checklists, i.e. a given checklist does not cover
    more than one site (this is TRUE for the four sites we consider in our data)
    
    df_in: eBird dataframe
    '''
    df = df_in.sort_values(by='observation_date').reset_index().copy() # Sort chronologically, reset the indices, and copy
    sites = df.Site_ID.unique() # List of sites
    for site in sites:
        dates = df.loc[df.Site_ID == site, 'observation_date'].unique() # All the unique observation dates
        # Handle the first observation, whose shifted column will be populated with nan
        df.loc[(df.Site_ID == site) & (df.observation_date == dates[0]), 'species_observed_prev'] = np.nan
        # Loop through the rest of the unique dates
        for i in range(1, len(dates)):
            current_date = dates[i]
            prev_date = dates[i-1]
            # Get the occupancy status from the previous observation. Note that we pull the array of unique values from 
            # the previous date and sum them. The possible unique arrays are True only, False only, or True and False. 
            # In the first two cases, summing the array gives us the same value as the array's element. In the third case
            # (which will happen if there are multiple different checklists on the same day and some of the observers
            # saw the species while others didn't), we'll take this to be true occupancy, so summing the array gives 
            # us True as desired.
            prev_occ = df.loc[(df.observation_date == prev_date) & (df.Site_ID == site), 'species_observed'].unique().sum()
            # Set the shifted column value for the current date to the previous date's occupancy status
            df.loc[(df.Site_ID == site) & (df.observation_date == current_date), 'species_observed_prev'] = prev_occ   
    return df

In [232]:
def get_all_shifts(df_in, year_start = 2012, year_stop = 2021):
    """Convenience function to get all the shifted features in a single step.
    df_in: eBird dataframe
    year_start: first year of data
    year_stop: last year of data
    returns: dataframe with new columns corresponding to all the temporally-shifted features."""
    
    df_out = occ_shift(monthly_cov_shift(yearly_cov_shift(df_in)))
    df_out = df_out.dropna() # Note that we drop the null values. That is, we drop the first step in our original
                             # 'time-series' since there was no prior data with which we could create a 
                             # shifted feature for that first step.
    df_out.reset_index(drop = True, inplace=True)
    return df_out

In [252]:
all_species = birds.scientific_name.unique()
all_species

array(['Arremon taciturnus', 'Bubulcus ibis', 'Bucco capensis',
       'Capito niger', 'Ceratopipra erythrocephala', 'Conopophaga aurita',
       'Crotophaga ani', 'Cyanocorax heilprini',
       'Euchrepomis spodioptila', 'Attila spadiceus',
       'Gymnopithys rufigula', 'Lipaugus vociferans',
       'Ramphastos tucanus', 'Rupornis magnirostris',
       'Thamnomanes caesius', 'Tinamus major', 'Trogon viridis',
       'Florisuga mellivora', 'Dendroplex picus', 'Thraupis palmarum',
       'Ramphocelus carbo', 'Pitangus sulphuratus', 'Cacicus cela',
       'Coragyps atratus'], dtype=object)

In [287]:
suffixes = ['arr_t', 'bub_i', 'buc_c','cap_n', 'cer_e', 'con_a','cro_a', 'cya_h','euc_s', 'att_s','gym_r',
            'lip_v','ram_t', 'rup_m','tha_c', 'tin_m', 'tro_v','flo_m', 'den_p', 'thr_p','ram_c', 'pit_s', 
            'cac_c','cor_a']

In [286]:
birds_arr_t = birds.loc[birds.scientific_name == all_species[0]].sort_values(by='observation_date').copy()
birds_bub_i = birds.loc[birds.scientific_name == all_species[1]].sort_values(by='observation_date').copy()
birds_buc_c = birds.loc[birds.scientific_name == all_species[2]].sort_values(by='observation_date').copy()
birds_cap_n = birds.loc[birds.scientific_name == all_species[3]].sort_values(by='observation_date').copy()
birds_cer_e = birds.loc[birds.scientific_name == all_species[4]].sort_values(by='observation_date').copy()
birds_con_a = birds.loc[birds.scientific_name == all_species[5]].sort_values(by='observation_date').copy()
birds_cro_a = birds.loc[birds.scientific_name == all_species[6]].sort_values(by='observation_date').copy()
birds_cya_h = birds.loc[birds.scientific_name == all_species[7]].sort_values(by='observation_date').copy()
birds_euc_s = birds.loc[birds.scientific_name == all_species[8]].sort_values(by='observation_date').copy()
birds_att_s = birds.loc[birds.scientific_name == all_species[9]].sort_values(by='observation_date').copy()
birds_gym_r = birds.loc[birds.scientific_name == all_species[10]].sort_values(by='observation_date').copy()
birds_lip_v = birds.loc[birds.scientific_name == all_species[11]].sort_values(by='observation_date').copy()
birds_ram_t = birds.loc[birds.scientific_name == all_species[12]].sort_values(by='observation_date').copy()
birds_rup_m = birds.loc[birds.scientific_name == all_species[13]].sort_values(by='observation_date').copy()
birds_tha_c = birds.loc[birds.scientific_name == all_species[14]].sort_values(by='observation_date').copy()
birds_tin_m = birds.loc[birds.scientific_name == all_species[15]].sort_values(by='observation_date').copy()
birds_tro_v = birds.loc[birds.scientific_name == all_species[16]].sort_values(by='observation_date').copy()
birds_flo_m = birds.loc[birds.scientific_name == all_species[17]].sort_values(by='observation_date').copy()
birds_den_p = birds.loc[birds.scientific_name == all_species[18]].sort_values(by='observation_date').copy()
birds_thr_p = birds.loc[birds.scientific_name == all_species[19]].sort_values(by='observation_date').copy()
birds_ram_c = birds.loc[birds.scientific_name == all_species[20]].sort_values(by='observation_date').copy()
birds_pit_s = birds.loc[birds.scientific_name == all_species[21]].sort_values(by='observation_date').copy()
birds_cac_c = birds.loc[birds.scientific_name == all_species[22]].sort_values(by='observation_date').copy()
birds_cor_a = birds.loc[birds.scientific_name == all_species[23]].sort_values(by='observation_date').copy()

In [289]:
# first consider just birds_lip_v
birds_lip_v_shifted = get_all_shifts(birds_lip_v)


In [290]:
birds_lip_v_shifted

,level_0,index,prec,tmax,tmin,EVI,umd_tree_cover_loss_ha,umd_tree_cover_loss_from_fires_ha,Trophic_Niche,Sp_Gen,...,scientific_name,species_observed,Site_ID,EVI_prev_yr,umd_tree_cover_loss_ha_prev_yr,umd_tree_cover_loss_from_fires_ha_prev_yr,tmax_prev_month,tmin_prev_month,prec_prev_month,species_observed_prev
0,124,3033,271.12,30.00,24.0,0.484687,126022.0434,12047.60718,Frugivore,forest_specialist,...,Lipaugus vociferans,False,69.0,0.464591,163604.2673,25449.43001,31.00,24.0,278.86,0.0
1,125,3058,271.12,30.00,24.0,0.484687,126022.0434,12047.60718,Frugivore,forest_specialist,...,Lipaugus vociferans,False,69.0,0.464591,163604.2673,25449.43001,31.00,24.0,278.86,0.0
2,126,3094,263.35,29.88,23.0,0.491057,126022.0434,12047.60718,Frugivore,forest_specialist,...,Lipaugus vociferans,False,79.0,0.472961,163604.2673,25449.43001,31.00,24.0,234.08,0.0
3,127,3108,263.35,29.88,23.0,0.491057,126022.0434,12047.60718,Frugivore,forest_specialist,...,Lipaugus vociferans,False,79.0,0.472961,163604.2673,25449.43001,31.00,24.0,234.08,0.0
4,128,3133,263.35,29.88,23.0,0.491057,126022.0434,12047.60718,Frugivore,forest_specialist,...,Lipaugus vociferans,False,79.0,0.472961,163604.2673,25449.43001,31.00,24.0,234.08,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3799,3920,94039,189.86,30.00,24.0,0.454878,313520.6806,42566.22017,Frugivore,forest_specialist,...,Lipaugus vociferans,False,79.0,0.508274,242978.0788,27399.03555,31.00,24.0,251.07,0.0
3800,3924,94051,189.86,30.00,24.0,0.454878,313520.6806,42566.22017,Frugivore,forest_specialist,...,Lipaugus vociferans,True,79.0,0.508274,242978.0788,27399.03555,31.00,24.0,251.07,0.0
3801,3926,94085,219.01,30.00,24.0,0.396152,313520.6806,42566.22017,Frugivore,forest_specialist,...,Lipaugus vociferans,False,69.0,0.485074,242978.0788,27399.03555,31.88,25.0,255.07,0.0
3802,3925,94071,219.01,30.00,24.0,0.396152,313520.6806,42566.22017,Frugivore,forest_specialist,...,Lipaugus vociferans,False,69.0,0.485074,242978.0788,27399.03555,31.88,25.0,255.07,0.0


Now to fit some models

In [336]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, precision_recall_curve
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.dummy import DummyClassifier
import xgboost as xgb

In [ ]:
features = ['effort_distance_km', 'number_observers', 'effort_hours', 'year', 'day_of_year', 'hours_of_day',
            'EVI_prev_yr','umd_tree_cover_loss_ha_prev_yr','umd_tree_cover_loss_from_fires_ha_prev_yr',
            'tmax_prev_month','tmin_prev_month', 'prec_prev_month', 'species_observed_prev']

In [317]:
# Baseline with sklearn's DummyClassifier (strategy = 'uniform' -> random guesses)
dummy = DummyClassifier(strategy = 'uniform')
dummy.fit(birds_lip_v_shifted[features], birds_lip_v_shifted.species_observed)
dummy_pred = dummy.predict(birds_lip_v_shifted[features])
print(classification_report(y_true = birds_lip_v_shifted.species_observed, y_pred = dummy_pred))

              precision    recall  f1-score   support

       False       0.84      0.51      0.63      3170
        True       0.17      0.50      0.25       634

    accuracy                           0.51      3804
   macro avg       0.50      0.51      0.44      3804
weighted avg       0.73      0.51      0.57      3804



In [324]:
# Baseline with sklearn's DummyClassifier (strategy = 'stratified' -> guesses stratified by class)
dummy = DummyClassifier(strategy = 'stratified')
dummy.fit(birds_lip_v_shifted[features], birds_lip_v_shifted.species_observed)
dummy_pred = dummy.predict(birds_lip_v_shifted[features])
print(classification_report(y_true = birds_lip_v_shifted.species_observed, y_pred = dummy_pred))

              precision    recall  f1-score   support

       False       0.83      0.83      0.83      3170
        True       0.17      0.17      0.17       634

    accuracy                           0.72      3804
   macro avg       0.50      0.50      0.50      3804
weighted avg       0.72      0.72      0.72      3804



In [326]:
rf = RandomForestClassifier(
    n_estimators = 500, # number of trees in ensemble
    max_depth = 10, # max_depth of each tree
    min_samples_leaf = 5, 
    bootstrap= True, # sampling with replacement
    #max_samples = 500, # number of training samples selected with replacement to build tree
    random_state = 216 # for consistency
    )
rf.fit(birds_lip_v_shifted[features], birds_lip_v_shifted.species_observed)
pred = rf.predict(birds_lip_v_shifted[features])
print(classification_report(y_true = birds_lip_v_shifted.species_observed, y_pred = pred))

              precision    recall  f1-score   support

       False       0.89      0.99      0.94      3170
        True       0.94      0.41      0.57       634

    accuracy                           0.90      3804
   macro avg       0.92      0.70      0.75      3804
weighted avg       0.90      0.90      0.88      3804



In [334]:
brf = BalancedRandomForestClassifier(n_estimators = 500, # number of trees in ensemble
    max_depth = 10, # max_depth of each tree
    min_samples_leaf = 5, 
    max_features = 2, # default is round(sqrt(num_features)), which in this case is 1.
    bootstrap= True, # sampling with replacement
    #max_samples = 500, # number of training samples selected with replacement to build tree
    random_state = 216, # for consistency
    replacement = True,
    sampling_strategy = 'auto',
    class_weight = 'balanced_subsample'
    )
brf.fit(birds_lip_v_shifted[features], birds_lip_v_shifted.species_observed)
pred = brf.predict(birds_lip_v_shifted[features])
print(classification_report(y_true = birds_lip_v_shifted.species_observed, y_pred = pred))

              precision    recall  f1-score   support

       False       0.98      0.73      0.83      3170
        True       0.41      0.94      0.57       634

    accuracy                           0.76      3804
   macro avg       0.69      0.83      0.70      3804
weighted avg       0.89      0.76      0.79      3804



In [338]:
# "Naive" xgboost classifier
xgbclass = xgb.XGBClassifier()
xgbclass.fit(birds_lip_v_shifted[features], birds_lip_v_shifted.species_observed)
pred = xgbclass.predict(birds_lip_v_shifted[features])
print(classification_report(y_true = birds_lip_v_shifted.species_observed, y_pred = pred))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00      3170
        True       1.00      0.98      0.99       634

    accuracy                           1.00      3804
   macro avg       1.00      0.99      0.99      3804
weighted avg       1.00      1.00      1.00      3804



In [350]:
# Balanced xgboost classifier
# minority/majority ratio 
minovermaj = birds_lip_v_shifted.species_observed.value_counts()[True]/birds_lip_v_shifted.species_observed.value_counts()[False]
# weighting parameter will be (1/minovermaj)
weight = 1/minovermaj
xgbclass = xgb.XGBClassifier(scale_pos_weight = weight)
xgbclass.fit(birds_lip_v_shifted[features], birds_lip_v_shifted.species_observed)
pred = xgbclass.predict(birds_lip_v_shifted[features])
print(classification_report(y_true = birds_lip_v_shifted.species_observed, y_pred = pred))

              precision    recall  f1-score   support

       False       1.00      0.99      1.00      3170
        True       0.96      1.00      0.98       634

    accuracy                           0.99      3804
   macro avg       0.98      1.00      0.99      3804
weighted avg       0.99      0.99      0.99      3804

